In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import os

In [ ]:
# Read the CSV into memory
df = pl.read_csv("/Users/ettoremiglioranza/Projects/gar4ds/data/pm10_era5_land_era5_reanalysis_blh_final.csv")

# Write the DataFrame to a Parquet file
df.write_parquet("../data/dataset.parquet", compression="zstd")

In [2]:
# Load the dataset
df = pl.read_parquet("../data/dataset.parquet")

# 1. Schema and Data Types
print("--- Schema ---")
print(df.schema)

# 2. Structural Summary (Rows, Columns, and Types)
print("\n--- Dataframe Summary ---")
print(df.glimpse())

# 3. Descriptive Statistics (Mean, Min, Max, Null counts)
print("\n--- Descriptive Statistics ---")
print(df.describe())

# 4. Memory Usage
print("\n--- Memory Usage (Bytes) ---")
print(df.estimated_size())

--- Schema ---
Schema({'Data': String, 'Stazione_APPA': String, 'PM10_(ug.m-3)': Float64, 'Vwind_550hPa': Float64, 'Vwind_850hPa': Float64, 'Vwind_950hPa': Float64, 'blh_mean_daily': Float64, 'Humidity_550hPa': Float64, 'Humidity_850hPa': Float64, 'Humidity_950hPa': Float64, 'Radiaz_Solare_tot_(kJ/m2)': Float64, 'Pressione_Atm_(hPa)': Float64, 'Temperatura_(°C)': Float64, 'Temperature_550hPa': Float64, 'Temperature_850hPa': Float64, 'Temperature_950hPa': Float64, 'Precipitazione_(mm)': Float64, 'Uwind_550hPa': Float64, 'Uwind_850hPa': Float64, 'Uwind_950hPa': Float64, 'Vel_Vento_media_(m/s)': Float64, 'Direzione_Vento_media_(°)': Float64, 'pm10_A22_sud,_Termeno,_c/o_Maso_Binnenland': Float64, 'pm10_Brunico_1,_c/o_parcheggio_di_via_Goethe': Float64, 'pm10_Bressanone,_c/o_Villa_Adele_in_Via_Stazione': Float64, 'pm10_Bolzano_4-Via_C.Augusta': Float64, 'pm10_Bolzano_5-P.zza_Adriano': Float64, 'pm10_Laces_1-Via_Stazione,_parcheggio_fs': Float64, 'pm10_Laives_1,_c/o_campi_sportivi_zona_Galiz

In [3]:
# 1. Pivot APPA Stations (Long to Wide)
# Isolates the 'Stazione_APPA' column and pivots it against the 'PM10' values.
appa_wide = (
    df.select(["Data", "Stazione_APPA", "PM10_(ug.m-3)"])
    .pivot(
        on="Stazione_APPA",
        index="Data",
        values="PM10_(ug.m-3)",
        aggregate_function="first"
    )
)

# 2. Extract External Stations (Deduplicate)
# Selects columns starting with 'pm10_' and retains distinct rows based on timestamp.
ext_cols = [c for c in df.columns if c.startswith("pm10_")]

external_wide = (
    df.select([pl.col("Data")] + [pl.col(c) for c in ext_cols])
    .unique(subset=["Data"])
    .rename({c: c.replace("pm10_", "") for c in ext_cols})
)

# 3. Merge and Sort
# Performs an outer join on the timestamp and sorts chronologically.
spatial_Y = (
    appa_wide.join(external_wide, on="Data", how="outer")
    .sort("Data")
)

# Output Verification
print(f"Shape: {spatial_Y.shape}")
print(spatial_Y.head())

Shape: (96432, 38)
shape: (5, 38)
┌─────────────┬─────────────┬───────┬────────────┬───┬────────┬────────────┬──────────┬────────────┐
│ Data        ┆ Borgo       ┆ Monte ┆ Parco S.   ┆ … ┆ Mansue ┆ TV_S_Agnes ┆ Bissuola ┆ VE_Tagliam │
│ ---         ┆ Valsugana   ┆ Gaza  ┆ Chiara     ┆   ┆ ---    ┆ e          ┆ ---      ┆ ento       │
│ str         ┆ ---         ┆ ---   ┆ ---        ┆   ┆ f64    ┆ ---        ┆ f64      ┆ ---        │
│             ┆ f64         ┆ f64   ┆ f64        ┆   ┆        ┆ f64        ┆          ┆ f64        │
╞═════════════╪═════════════╪═══════╪════════════╪═══╪════════╪════════════╪══════════╪════════════╡
│ 2014-01-01  ┆ 2.0         ┆ 38.0  ┆ 23.0       ┆ … ┆ 58.0   ┆ 199.0      ┆ 170.0    ┆ 172.0      │
│ 00:00:00    ┆             ┆       ┆            ┆   ┆        ┆            ┆          ┆            │
│ 2014-01-01  ┆ 3.0         ┆ 41.0  ┆ 23.0       ┆ … ┆ 58.0   ┆ 199.0      ┆ 170.0    ┆ 172.0      │
│ 01:00:00    ┆             ┆       ┆            ┆   ┆   

/var/folders/j4/lw9rt9517k134dz28rgq71r40000gn/T/ipykernel_41954/2962658562.py:26: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  appa_wide.join(external_wide, on="Data", how="outer")


In [ ]:
# Create data directory if it doesn't exist
output_dir = "../data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1. Pivot APPA Stations (Long -> Wide)
# Each unique timestamp becomes a row; each Stazione_APPA becomes a column.
appa_wide = (
    df.select(["Data", "Stazione_APPA", "PM10_(ug.m-3)"])
    .pivot(
        on="Stazione_APPA",
        index="Data",
        values="PM10_(ug.m-3)",
        aggregate_function="first"
    )
)

# 2. Extract External Stations (Deduplicate Wide Columns)
# Isolate columns starting with 'pm10_'. Since these repeat for every APPA row,
# we group by 'Data' and take the first value to eliminate redundancy.
ext_cols = [c for c in df.columns if c.lower().startswith("pm10_")]

external_wide = (
    df.select([pl.col("Data")] + [pl.col(c) for c in ext_cols])
    .unique(subset=["Data"])
    .rename({c: c.replace("pm10_", "").replace("PM10_", "") for c in ext_cols})
)

# 3. Merge into a Single Spatial Matrix
# Outer join ensures we keep all timestamps from both sources
spatial_Y = (
    appa_wide.join(external_wide, on="Data", how="full")
    .sort("Data")
)

# 4. Save to Parquet
output_path = os.path.join(output_dir, "spatial_pollution_matrix.parquet")
spatial_Y.write_parquet(output_path)

print(f"Spatial Matrix saved to: {output_path}")
print(f"Dimensions: {spatial_Y.shape}")
print(spatial_Y.head())

Spatial Matrix saved to: ./data/spatial_pollution_matrix.parquet
Dimensions: (96432, 39)
shape: (5, 39)
┌─────────────┬─────────────┬───────┬────────────┬───┬────────┬────────────┬──────────┬────────────┐
│ Data        ┆ Borgo       ┆ Monte ┆ Parco S.   ┆ … ┆ Mansue ┆ TV_S_Agnes ┆ Bissuola ┆ VE_Tagliam │
│ ---         ┆ Valsugana   ┆ Gaza  ┆ Chiara     ┆   ┆ ---    ┆ e          ┆ ---      ┆ ento       │
│ str         ┆ ---         ┆ ---   ┆ ---        ┆   ┆ f64    ┆ ---        ┆ f64      ┆ ---        │
│             ┆ f64         ┆ f64   ┆ f64        ┆   ┆        ┆ f64        ┆          ┆ f64        │
╞═════════════╪═════════════╪═══════╪════════════╪═══╪════════╪════════════╪══════════╪════════════╡
│ 2014-01-01  ┆ 2.0         ┆ 38.0  ┆ 23.0       ┆ … ┆ 58.0   ┆ 199.0      ┆ 170.0    ┆ 172.0      │
│ 00:00:00    ┆             ┆       ┆            ┆   ┆        ┆            ┆          ┆            │
│ 2014-01-01  ┆ 3.0         ┆ 41.0  ┆ 23.0       ┆ … ┆ 58.0   ┆ 199.0      ┆ 170.0    ┆ 

In [ ]:
# Lock (select) the first 10 columns and print the head
first_10_cols = spatial_Y.columns[:10]
print(spatial_Y.select(first_10_cols).head())